In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from statsbombpy import sb
import traceback

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")

In [2]:
competitions = sb.competitions()
# selected_competitions is where match_available_360 is not None
#selected_competitions = competitions[competitions['match_available_360'].notnull()]

# drop season_id = 90 row from selected_competitions
# selected_competitions = selected_competitions[selected_competitions['season_id'] != 90]

# selected_competitions is where competition_gender is male
selected_competitions_male = competitions[competitions['competition_gender'] == 'male']
selected_competitions = selected_competitions_male[selected_competitions_male['season_id'] != 76]
# selected_competitions = competitions

In [2]:
def get_fouls(match_ids_list, columns, subset):
  """
  This method gets all the fouls from a list of match IDs.

  Args:
    match_ids_list: A list of match IDs.
    columns: A list of the columns that are interesting.
    subset: A list of the columns that should not be NaN.

  Returns:
    A DataFrame of all the fouls.
  """

  fouls = pd.DataFrame()

  for match_id in tqdm(match_ids_list):
    match_events_df = sb.events(match_id=match_id)
    match_events_df_fouls = match_events_df.reindex(columns=columns)

    match_events_df_fouls.dropna(subset=subset, how='all', inplace=True)

    fouls = pd.concat([fouls, match_events_df_fouls], ignore_index=True)

  return fouls

In [3]:
def get_match_ids(row):
    try:
        return sb.matches(row.competition_id, row.season_id).match_id.tolist()
    except Exception as e:
        traceback.print_exc()
        print("Error in row:")
        print(row)
        return []

cmps = sb.competitions()

# # save competion_id and season_id in cmps where match_available_360 is not None
# cmps = cmps[cmps.competition_gender == 'male']

# # drop season_id = 90 row from cmps
# # cmps = cmps[cmps.season_id != 90]

# cmps = cmps[['competition_id', 'season_id', 'competition_name', 'season_name']]

# # remove season_id = 76 from cmps
# cmps = cmps[cmps.season_id != 76]

# # add column 'match_ids' to cmps. you can fill using sb.matches(row.competition_id, row.season_id). this should be a list
# cmps['match_ids'] = cmps.apply(lambda row: get_match_ids(row), axis=1)

# # add column 'match_count' to cmps
# cmps['match_count'] = cmps.apply(lambda row: len(row.match_ids), axis=1)

In [46]:
# # print total rows in cmps
# print(len(cmps))

# # print rows where match_count is atleast 10. do not print match_ids column
# cmps[cmps.match_count >= 10].drop(columns=['match_ids'])

# # print rows where match_count is greater than 40
# cmps[cmps.match_count > 40]

59


,competition_id,season_id,competition_name,season_name,match_ids,match_count
0,9,27,1. Bundesliga,2015/2016,"[3890561, 3890505, 3890511, 3890515, 3890411, ...",306
26,43,106,FIFA World Cup,2022,"[3857256, 3869151, 3857257, 3857258, 3857288, ...",64
27,43,3,FIFA World Cup,2018,"[7585, 7570, 7586, 7557, 7542, 8656, 7554, 755...",64
34,1238,108,Indian Super league,2021/2022,"[3827767, 3827335, 3827336, 3827338, 3827337, ...",115
40,11,27,La Liga,2015/2016,"[3825848, 3825895, 3825894, 3825855, 3825908, ...",380
55,7,27,Ligue 1,2015/2016,"[3901178, 3901260, 3829470, 3901259, 3901263, ...",377
58,2,27,Premier League,2015/2016,"[3754058, 3754245, 3754136, 3754037, 3754039, ...",380
60,12,27,Serie A,2015/2016,"[3879863, 3879773, 3879847, 3879862, 3879817, ...",380
62,55,43,UEFA Euro,2020,"[3795108, 3788769, 3788766, 3795220, 3788761, ...",51


In [57]:
# interesting_columns = ['foul_committed_advantage','foul_committed_offensive', 'foul_committed_penalty','foul_committed_card','foul_committed_counterpress','foul_committed_type', 'foul_won_advantage', 'foul_won_defensive', 'foul_won_penalty', 'id', 'index', 'location', 'match_id', 'minute', 'second', 'period', 'player', 'player_id', 'position', 'possession', 'possession_team', 'possession_team_id', 'team', 'team_id', 'timestamp', 'type']
# interesting_subset = ['foul_committed_advantage','foul_committed_card','foul_committed_offensive','foul_committed_penalty','foul_committed_counterpress','foul_committed_type']

# for index, cmp in cmps.iterrows():
#     # Call get_fouls() for each match in cmps. get_fouls() takes the match_ids list as the first argument.
#     all_fouls = get_fouls(cmp["match_ids"], interesting_columns, interesting_subset)
    
#     # Get count of each value in column 'foul_committed_card' and add this to cmps DataFrame. Save count of yellow cards in column 'yellow_card_count' and count of red cards in column 'red_card_count' and count of second yellow cards in column 'second_yellow_card_count'. Create these columns in cmps DataFrame if they don't exist.
#     yellow_card_count = all_fouls['foul_committed_card'].value_counts().get('Yellow Card', 0)
#     red_card_count = all_fouls['foul_committed_card'].value_counts().get('Red Card', 0)
#     second_yellow_card_count = all_fouls['foul_committed_card'].value_counts().get('Second Yellow', 0)
    
#     cmps.loc[index, 'yellow_card_count'] = yellow_card_count
#     cmps.loc[index, 'red_card_count'] = red_card_count
#     cmps.loc[index, 'second_yellow_card_count'] = second_yellow_card_count

100%|██████████| 3/3 [00:01<00:00,  1.51it/s]


In [5]:
# cmps.head(10)
# save cmps to pickle file
# cmps.to_pickle('cmps.pkl')

# load cmps from pickle file
cmps = pd.read_pickle('cmps.pkl')
cmps.head()

# drop season_id = 76 row from cmps
cmps = cmps[cmps.season_id != 76]

In [15]:
def get_match_ids_info(competitions):
  """
  Get the match IDs and information for each match in the selected competitions.

  Args:
    competitions: A pandas DataFrame of selected competitions.

  Returns:
    A tuple of two lists: `match_ids_list` and `match_ids_info`.
  """

  match_ids_list = []
  match_ids_info = {}

  for index, row in competitions.iterrows():
    try:
      matches = sb.matches(row.competition_id, row.season_id)
    except Exception as e:
      print("Error in row:")
      print(row)
      print(e)
      exit()

    for match in matches.itertuples():
      match_ids_list.append(match.match_id)
      match_ids_info[match.match_id] = [match.home_team, match.away_team]

  return match_ids_list, match_ids_info

In [16]:
interesting_columns = ['foul_committed_advantage','foul_committed_offensive', 'foul_committed_penalty','foul_committed_card','foul_committed_counterpress','foul_committed_type', 'foul_won_advantage', 'foul_won_defensive', 'foul_won_penalty', 'id', 'index', 'location', 'match_id', 'minute', 'second', 'period', 'player', 'player_id', 'position', 'possession', 'possession_team', 'possession_team_id', 'team', 'team_id', 'timestamp', 'type']
interesting_subset = ['foul_committed_advantage','foul_committed_card','foul_committed_offensive','foul_committed_penalty','foul_committed_counterpress','foul_committed_type']

match_ids_list, match_ids_info = get_match_ids_info(selected_competitions)
# all_fouls = get_fouls(match_ids_list, interesting_columns, interesting_subset)

In [6]:
#all_fouls.to_pickle('all_fouls.pkl')

# load all_fouls from pickle file
all_fouls = pd.read_pickle('all_fouls.pkl')

In [7]:
all_fouls.head(2)

,foul_committed_advantage,foul_committed_offensive,foul_committed_penalty,foul_committed_card,foul_committed_counterpress,foul_committed_type,foul_won_advantage,foul_won_defensive,foul_won_penalty,id,index,location,match_id,minute,second,period,player,player_id,position,possession,possession_team,possession_team_id,team,team_id,timestamp,type
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f1dd3418-5813-4df3-b319-97e1f821a4bb,200,"[52.0, 55.2]",3890561,3,24,1,Leroy Sané,3053.0,Right Midfield,7,Hoffenheim,175,Schalke 04,181,00:03:24.672,Foul Committed
1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b9948ea0-9079-4d01-bdc7-8022567fe66a,1472,"[49.7, 2.1]",3890561,34,50,1,Jean-Eric Maxim Choupo-Moting,3499.0,Left Midfield,62,Hoffenheim,175,Schalke 04,181,00:34:50.508,Foul Committed


In [8]:
len(all_fouls)

24639

In [9]:
def get_goals(match_ids):
  """
  Get all goals from a list of match IDs.

  Args:
    match_ids: A list of match IDs.

  Returns:
    A DataFrame of goals and match events
  """

  goals = pd.DataFrame()
  match_events = pd.DataFrame()

  for match_id in tqdm(match_ids):
    match_events_df = sb.events(match_id=match_id)
    match_events = pd.concat([match_events, match_events_df], ignore_index=True)

    match_events_df_shots = match_events_df[match_events_df.type == 'Shot']
    match_events_df_goals = match_events_df_shots[match_events_df_shots.shot_outcome == 'Goal']

    match_events_df_goals['seconds'] = match_events_df_goals['minute'] * 60 + match_events_df_goals['second']
    goals = pd.concat([goals, match_events_df_goals], ignore_index=True)

  return goals, match_events

In [10]:
# all_goals, match_events = get_goals(match_ids_list)

# all_goals.to_pickle('all_goals.pkl')
# match_events.to_pickle('match_events.pkl')

# load all_goals and match_events from pickle file
all_goals = pd.read_pickle('all_goals.pkl')
match_events = pd.read_pickle('match_events.pkl')

In [11]:
def get_scoreline(goals, match_events, match_id, seconds):
    match_goals = goals[goals['match_id'] == match_id]
    
    # find unique team_id
    teams = match_events['team'].unique()
    match_goals = match_goals[match_goals['seconds'] <= seconds]

    # find goal scored by each_team
    team_goal_count = match_goals.groupby('team')['team'].count()
    team_goal_dict = {}
    
    for team in teams:
        # if team_goal_count['team'] is NaN, then add to team_goal_dict with key as team_name and value as 0
        if team not in team_goal_count:
            team_goal_dict[team] = 0
        else:
            team_goal_dict[team] = team_goal_count[team]
            
    return team_goal_dict


def distance_to_goal(isHomeTeam, location):
    if isHomeTeam:
        # calculate distance from location to away goal
        return np.sqrt((120 - location[0])**2 + (40 - location[1])**2)
    else:
        # calculate distance from location to home goal
        return np.sqrt((0 - location[0])**2 + (40 - location[1])**2)

def angle_to_goal(isHomeTeam, location):
    if isHomeTeam:
        # calculate angle from location to away goal
        if (120 - location[0]) != 0:
            return np.arctan((40 - location[1]) / (120 - location[0]))
        else:
            # handle denominator is 0 
            return np.pi / 2  # or any other appropriate value
    else:
        # calculate angle from location to home goal
        if (0 - location[0]) != 0:
            return np.arctan((40 - location[1]) / (0 - location[0]))
        else:
            # handle denominator is 0
            return np.pi / 2  # or any other appropriate value

In [12]:
# Write method to calculate previous foul count of the player making the foul. Inputs will be match_id, player_id, timestamp
def previous_foul_count(fouls, match_id, player_id, seconds):
    # get all fouls in the match
    match_fouls_df = fouls[fouls['match_id'] == match_id]
    
    # get all fouls by player_id
    player_fouls_df = match_fouls_df[match_fouls_df['player_id'] == player_id]
    
    player_fouls_df['seconds'] = player_fouls_df['minute'] * 60 + player_fouls_df['second']

    # get all fouls before timestamp
    previous_fouls_df = player_fouls_df[player_fouls_df['seconds'] <= seconds]
    
    # return count of fouls
    return len(previous_fouls_df)


# Write method to calculate previous foul count of the team making the foul. Inputs will be match_id, team_id, timestamp
def previous_foul_count_team(fouls, match_id, team_id, seconds):
    # get all fouls in the match
    match_fouls_df = fouls[fouls['match_id'] == match_id]
    
    # get all fouls by team_id
    team_fouls_df = match_fouls_df[match_fouls_df['team_id'] == team_id]
    
    team_fouls_df['seconds'] = team_fouls_df['minute'] * 60 + team_fouls_df['second']

    # get all fouls before timestamp
    team_fouls_df = team_fouls_df[team_fouls_df['seconds'] <= seconds]
    
    # return count of fouls
    return len(team_fouls_df)

In [13]:
def make_advanced_fouls(fouls, goals, match_events, match_ids_info):
  """
  This function takes a DataFrame of fouls and a `match_ids_info` DataFrame and returns a new DataFrame with advanced foul metrics.

  Args:
    all_fouls: A DataFrame of fouls.
    match_ids_info: A DataFrame that contains information about the matches, such as the home team and the away team.

  Returns:
    A DataFrame of advanced fouls.
  """

  # Copy the original DataFrame.
  fouls_advanced = fouls.copy()

  fouls_advanced['seconds_till_now'] = fouls_advanced['minute'] * 60 + fouls_advanced['second']
  fouls_advanced['scoreline_till_now'] = fouls_advanced.apply(lambda x: get_scoreline(goals, match_events, x['match_id'], x['seconds_till_now']), axis=1)
  fouls_advanced['distance_to_goal'] = fouls_advanced.apply(lambda x: distance_to_goal(x['team'] == match_ids_info[x['match_id']][0], x['location']), axis=1)
  fouls_advanced['angle_to_goal'] = fouls_advanced.apply(lambda x: angle_to_goal(x['team'] == match_ids_info[x['match_id']][0], x['location']), axis=1)

  # Calculate the number of fouls committed by the player up to the time of the foul.
  # all_fouls_advanced['foul_count_player_till_now'] = all_fouls_advanced.apply(lambda x: previous_foul_count(all_fouls, x['match_id'], x['player_id'], x['seconds_till_now']), axis=1)

  # Calculate the number of fouls committed by the team up to the time of the foul.
  # all_fouls_advanced['foul_count_team_till_now'] = all_fouls_advanced.apply(lambda x: previous_foul_count_team(all_fouls, x['match_id'], x['team_id'], x['seconds_till_now']), axis=1)

  return fouls_advanced

In [17]:
all_fouls_advanced = make_advanced_fouls(all_fouls, all_goals, match_events, match_ids_info)
all_fouls_advanced['foul_count_player_till_now'] = all_fouls_advanced.apply(lambda x: previous_foul_count(all_fouls, x['match_id'], x['player_id'], x['seconds_till_now']), axis=1)
all_fouls_advanced['foul_count_team_till_now'] = all_fouls_advanced.apply(lambda x: previous_foul_count_team(all_fouls, x['match_id'], x['team_id'], x['seconds_till_now']), axis=1)

all_fouls_advanced.to_pickle('all_fouls_advanced.pkl')


# load all_fouls_advanced from pickle file
# all_fouls_advanced = pd.read_pickle('all_fouls_advanced.pkl')

In [ ]:
import numpy as np

type_dict = {'Handball': 0, 'Dangerous Play': 1, 'Foul Out': 2, 'Dive': 3, '6 Seconds': 4, 'Backpass Pick': 5}

card_dict = {'Yellow Card': 0, 'Second Yellow': 0, 'Red Card': 1}

def create_features(team_fouls_df):
    all_features = []
    all_labels = []

    for _, row in team_fouls_df.iterrows():
        # select features only where foul_committed_type is not in type_dict keys
        if row['foul_committed_type'] in type_dict.keys():
            continue
        
        team_making_foul = row['team']

        scoreline = row['scoreline_till_now']
        team_not_making_foul = [team for team in scoreline.keys() if team != team_making_foul][0]
        
        score_difference_till_now = scoreline[team_not_making_foul] - scoreline[team_making_foul]

        features = [row['minute'], score_difference_till_now, row['distance_to_goal'], row['angle_to_goal'], row['foul_count_player_till_now'], row['foul_count_team_till_now'], row['id']]

        all_features.append(features)
        all_labels.append([card_dict.get(row['foul_committed_card'], 2)])
    
    return all_features, all_labels

In [ ]:
import pickle

all_the_features = []
all_the_labels = []

# use tqdm to iterate match_ids_list
for match_id in tqdm(match_ids_list):
    # get all fouls in the match
    match_fouls_df = all_fouls_advanced[all_fouls_advanced['match_id'] == match_id]

    feature, label = create_features(match_fouls_df)

    # append features and labels to all_the_features and all_the_labels
    all_the_features.extend(feature)
    all_the_labels.extend(label)

# save all_the_features and all_the_labels to files using pickle
with open('all_the_features.pkl', 'wb') as f:
    pickle.dump(all_the_features, f)

with open('all_the_labels.pkl', 'wb') as f:
    pickle.dump(all_the_labels, f)

In [ ]:
print (len(all_the_features))

# first 10 items in all_the_features and all_the_labels
print (all_the_features[:100])
print (all_the_labels[:100])

# print count of unique labels
print (np.unique(all_the_labels, return_counts=True))

In [ ]:
# save match_ids_list to file using pickle
with open('match_ids_list.pkl', 'wb') as f:
    pickle.dump(match_ids_list, f)